# Feature Engineering and Linear Regression - Tennis Demand

#### Table of Contents
1. [Imports](#imports)
2. [Data Import and Feature Engineering](#data-import-and-feature-engineering)
3. [MLFlow Experiments](#mlflow-experiments)

#### Imports

In [1]:
import mlflow  # ML experiment tracking
import pandas as pd  # Data manipulation

# Sklearn
from sklearn.model_selection import train_test_split  # Data splitting
from imblearn.over_sampling import SMOTE  # Handle class imbalance

from sklearn.linear_model import Ridge  # Ridge regression model
from sklearn.preprocessing import StandardScaler  # Feature scaling

In [2]:
# Local Modules

import os

os.sys.path.append("../src")

from process import *

#### Data Import and Feature Engineering

In [3]:
# Import data

data_path = "../data/processed/tennis_merged.csv"
df_tennis = pd.read_csv(data_path)

In [4]:
df_tennis.head(5)

,product_price,product_original_price,product_star_rating,product_num_ratings,product_minimum_offer_price,is_prime,climate_pledge_friendly,has_variations,coupon_discount,sales_volume
0,44.52,80.0,4.6,709,44.52,1.0,0.0,1.0,0.0,429
1,42.50,75.0,4.7,2028,42.50,1.0,0.0,1.0,0.0,117
2,38.50,70.0,4.5,502,38.50,1.0,0.0,1.0,0.0,185
3,30.69,50.0,4.5,40996,30.69,1.0,0.0,1.0,0.0,110
4,60.00,80.0,4.6,397,60.00,1.0,0.0,1.0,0.0,144


In [6]:
df_tennis_modified = df_tennis.copy()

In [8]:
df_tennis_modified["sum_ratings"] = df_tennis_modified["product_num_ratings"] * df_tennis_modified["product_star_rating"]
df_tennis_modified["absolute_discount"] = df_tennis_modified["product_price"] - df_tennis_modified["product_minimum_offer_price"]
df_tennis_modified["product_value"] = df_tennis_modified["product_price"] * df_tennis_modified["product_star_rating"]
df_tennis_modified["diversity_satisfaction"] = df_tennis_modified["product_star_rating"] * df_tennis_modified["has_variations"]

In [9]:
X = df_tennis.drop(labels=["sales_volume"], axis=1)
y = df_tennis[["sales_volume"]]

X_train_raw, X_test, y_train_raw, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_raw, y_train_raw, train_size=0.66)

#### MLFlow Experiments

In [12]:
amazon_mlflow_client = mlflow.MlflowClient(
    tracking_uri="http://127.0.0.1:8000"
)

mlflow.set_tracking_uri("http://127.0.0.1:8000")

In [13]:
run_name ="linear-regression-with-feature-engineering"
linear_models_experiment_id = '779742125521330833'
amazon_mlflow_client.create_run(
    experiment_id=linear_models_experiment_id,
    run_name=run_name,
    tags={
        "Dataset": "Amazon Tennis Demand - V2 (Feature Engineering)"
    }
)

<Run: data=<RunData: metrics={}, params={}, tags={'Dataset': 'Amazon Tennis Demand - V2 (Feature Engineering)',
 'mlflow.runName': 'linear-regression-with-feature-engineering'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/779742125521330833/b7a767b699d548efb6beba2262173f2d/artifacts', end_time=None, experiment_id='779742125521330833', lifecycle_stage='active', run_id='b7a767b699d548efb6beba2262173f2d', run_name='linear-regression-with-feature-engineering', run_uuid='b7a767b699d548efb6beba2262173f2d', start_time=1728582308969, status='RUNNING', user_id='unknown'>, inputs=<RunInputs: dataset_inputs=[]>>

In [17]:
ridge_params = {'alpha': 0.5}

linreg = Ridge(**ridge_params)
linreg.fit(X_train, y_train)

Ridge(alpha=0.5)

In [18]:
y_pred = linreg.predict(X_val)

In [23]:
linreg_performance = evaluate_regression_model(
    model=linreg,
    y_pred=y_pred,
    y_true=y_val
)

12:49:45.605 | INFO    | Task run 'evaluate_regression_model' - Created task run 'evaluate_regression_model' for task 'evaluate_regression_model'

12:49:45.688 | INFO    | Task run 'evaluate_regression_model' - Finished in state Completed()

In [28]:
with mlflow.start_run(
    experiment_id=linear_models_experiment_id,
    run_name=run_name
) as run:
    mlflow.log_params(ridge_params)
    mlflow.log_metrics(linreg_performance)
    mlflow.sklearn.log_model(
        sk_model=linreg,
        input_example=X_val,
        artifact_path="ridge-regression-with-feature-engineering"
    )    

/home/santi/current-projects/public-apis/amazon-mlops-dashboard/venv/lib/python3.11/site-packages/mlflow/types/utils.py:407: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2024/10/10 12:56:30 INFO mlflow.tracking._tracking_service.client: 🏃 View run linear-regression-with-feature-engineering at: http://127.0.0.1:8000/#/experiments/779742125521330833/runs/a8c2d

What if we standarize the data?

In [30]:
scaler = StandardScaler()

X_train_std = scaler.fit_transform(X_train)
X_val_std = scaler.transform(X_val)

In [32]:
ridge_reg = Ridge(**ridge_params)

ridge_reg.fit(X_train_std, y_train)
y_pred = ridge_reg.predict(X_val_std)


In [34]:
# Evaluate performance

ridge_reg_performance = evaluate_regression_model(
    model=ridge_reg,
    y_pred=y_pred,
    y_true=y_val
)

13:04:04.650 | INFO    | Task run 'evaluate_regression_model' - Created task run 'evaluate_regression_model' for task 'evaluate_regression_model'

13:04:05.648 | INFO    | Task run 'evaluate_regression_model' - Finished in state Completed()

In [35]:
std_run_name = "ridge-std-feature-engineering"

amazon_mlflow_client.create_run(
    experiment_id=linear_models_experiment_id,
    run_name=std_run_name
)

<Run: data=<RunData: metrics={}, params={}, tags={'mlflow.runName': 'ridge-std-feature-engineering'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/779742125521330833/acd03802903f40ba929391f7c442e41f/artifacts', end_time=None, experiment_id='779742125521330833', lifecycle_stage='active', run_id='acd03802903f40ba929391f7c442e41f', run_name='ridge-std-feature-engineering', run_uuid='acd03802903f40ba929391f7c442e41f', start_time=1728583541421, status='RUNNING', user_id='unknown'>, inputs=<RunInputs: dataset_inputs=[]>>

In [36]:
with mlflow.start_run(
    experiment_id=linear_models_experiment_id,
    run_name=std_run_name
) as run:
    mlflow.log_params(ridge_params)
    mlflow.log_metrics(ridge_reg_performance)
    mlflow.sklearn.log_model(
        sk_model=ridge_reg,
        input_example=X_val_std,
        artifact_path="ridge-with-standar-scaler-and-feature-engineering"
    )

2024/10/10 13:08:41 INFO mlflow.tracking._tracking_service.client: 🏃 View run ridge-std-feature-engineering at: http://127.0.0.1:8000/#/experiments/779742125521330833/runs/d444ea8e4032433385b7bf3d692e326e.
2024/10/10 13:08:41 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8000/#/experiments/779742125521330833.


14:38:18.481 | WARNING | prefect.server.services.marklateruns - MarkLateRuns took 16.49266 seconds to run, which is longer than its loop interval of 5.0 seconds.
14:38:19.337 | WARNING | prefect.server.services.recentdeploymentsscheduler - RecentDeploymentsScheduler took 15.349286 seconds to run, which is longer than its loop interval of 5 seconds.
14:38:20.487 | WARNING | prefect.server.services.failexpiredpauses - FailExpiredPauses took 16.775007 seconds to run, which is longer than its loop interval of 5.0 seconds.
14:38:32.731 | WARNING | prefect.server.services.flowrunnotifications - FlowRunNotifications took 28.566165 seconds to run, which is longer than its loop interval of 4 seconds.
14:39:08.699 | WARNING | prefect.server.services.recentdeploymentsscheduler - RecentDeploymentsScheduler took 48.41631 seconds to run, which is longer than its loop interval of 5 seconds.
14:39:16.788 | WARNING | prefect.server.services.marklateruns - MarkLateRuns took 57.369331 seconds to run, whi